# 실습과제 2(쇼핑몰 데이터 분석 미니 프로젝트)

### 데이터 준비 및 가공

In [ ]:
import pandas as pd
df = pd.read_csv('./미니프로젝트-쇼핑몰 실습데이터 - 2019-01-01 ~ 2019-12-31.csv')

In [ ]:
df.dtypes

In [ ]:
df['업체명'] = df['업체명'].replace(to_replace='.*지니.*', value='지니', regex=True)
df['결제방법'] = df['결제방법'].replace(to_replace='신.*', value='신용카드', regex=True)
df['결제방법'] = df['결제방법'].replace(to_replace='후.*', value='후불', regex=True)
df['결제방법'] = df['결제방법'].replace(to_replace='정.*', value='정기결제', regex=True)
df['결제방법'] = df['결제방법'].replace(to_replace='적.*', value='적립금', regex=True)
df['결제방법'] = df['결제방법'].replace(to_replace='현.*', value='현금간편결제', regex=True)

In [ ]:
df['판매금액'].fillna(0)
df['판매금액'] = df['판매금액'].str.replace(',', '')
df['판매금액'] = df['판매금액'].astype(int)  #문자열이었던 것을 정수형으로 바꿨음

In [ ]:
df = df[df['업체명'].isin(['지니','다우기술','천재태블릿'])]

In [ ]:
from datetime import datetime

def convert_to_date(date_str):
    return datetime.strptime(date_str, '%Y-%m-%d')

df['주문일자'] = df['주문일자'].apply(convert_to_date)
df['주문수량'] = df['주문수량'].fillna(0).astype(int)
df['판매금액'] = df['판매금액'].fillna(0).astype(int)
df = df.drop('초도상품', axis=1)
df = df.drop('제작문구 내역', axis=1)

In [ ]:
print(df.dtypes)

In [ ]:
df['처리상태'].unique()

In [ ]:
df['순이익'] = df.apply(lambda row: row['판매금액'] if row['처리상태'] in ['교환완료', '구매확정']
                     else row['판매금액'] / 2 if row['처리상태'] in ['부분환불', '부분취소']
                     else 0, axis=1).astype(int)

In [ ]:
df['할부기간'] = df['할부기간'].str.replace('개월', '').fillna(0).astype(int)
df['할부기간'].unique()

In [ ]:
# 후불기간이 존재할 경우 순이익에서 나누어 그 다음달부터  n개월치 만큼 각각의 월에 부여할 수 있도록 한다.

In [ ]:
df['주문월'] = pd.to_datetime(df['주문일자']).dt.month
df['주문년도'] = pd.to_datetime(df['주문일자']).dt.year
df1= df[df['업체명'] == '지니']
df2= df[df['업체명'] == '다우기술']
df3= df[df['업체명'] == '천재태블릿']

In [ ]:
print(df.dtypes)

## 1번

### 가장 많은 거래가 이루어지고 있는 3개의 입점 기업을 선정

In [ ]:
df.groupby('업체명').agg({'판매금액': 'sum'}).sort_values(by= '판매금액',ascending = False)

In [ ]:
df.groupby('업체명').agg({'주문수량': 'sum'}).sort_values(by= '주문수량',ascending = False)

In [ ]:
len(df[df['업체명'] == '천재태블릿'])

### 22/11/08일을 기준으로 최근 3달을 바탕으로 1달씩 구매확정 비율

In [ ]:
df_1 = df[(df['주문일자'] >= convert_to_date('2022-8-9')) & (df['주문일자'] <= convert_to_date('2022-9-8'))]
df_2 = df_1.groupby('업체명').agg({'처리상태' : 'count'})
df_2['구매확정'] = df_1[(df_1['처리상태'] == '구매확정')].groupby('업체명').agg({'처리상태' : 'count'})
df_2['확정비율'] = round((df_2['구매확정']/df_2['처리상태'])*100,2)
df_2['미결제건'] = df_1[(df_1['처리상태'] == '미결제')].groupby('업체명').agg({'처리상태' : 'count'})
df_2['미결제비율'] = round((df_2['미결제건']/df_2['처리상태'])*100,2)
df_2.rename(columns={'처리상태': '주문건수'}, inplace=True)
df_2

In [ ]:
df_1 = df[(df['주문일자'] >= convert_to_date('2022-9-9')) & (df['주문일자'] <= convert_to_date('2022-10-8'))]
df_2 = df_1.groupby('업체명').agg({'처리상태' : 'count'})
df_2['구매확정'] = df_1[(df_1['처리상태'] == '구매확정')].groupby('업체명').agg({'처리상태' : 'count'})
df_2['확정비율'] = round((df_2['구매확정']/df_2['처리상태'])*100,2)
df_2['미결제건'] = df_1[(df_1['처리상태'] == '미결제')].groupby('업체명').agg({'처리상태' : 'count'})
df_2['미결제비율'] = round((df_2['미결제건']/df_2['처리상태'])*100,2)
df_2.rename(columns={'처리상태': '주문건수'}, inplace=True)
df_2

In [ ]:
df_1 = df[(df['주문일자'] >= convert_to_date('2022-10-9')) & (df['주문일자'] <= convert_to_date('2022-11-8'))]
df_2 = df_1.groupby('업체명').agg({'처리상태' : 'count'})
df_2['구매확정'] = df_1[(df_1['처리상태'] == '구매확정')].groupby('업체명').agg({'처리상태' : 'count'})
df_2['확정비율'] = round((df_2['구매확정']/df_2['처리상태'])*100,2)
df_2['미결제건'] = df_1[(df_1['처리상태'] == '미결제')].groupby('업체명').agg({'처리상태' : 'count'})
df_2['미결제비율'] = round((df_2['미결제건']/df_2['처리상태'])*100,2)
df_2.rename(columns={'처리상태': '주문건수'}, inplace=True)
df_2

### 2022년 기준 입점업체 월별 거래 빈도 확인

In [ ]:
df_3 = df[(df['주문일자'] >= convert_to_date('2022-01-01'))]
df_4 = df_3[df_3['업체명']== '지니']
df_5 = df_3[df_3['업체명']== '다우기술']
df_6 = df_3[df_3['업체명']== '천재태블릿']

#### 지니 회사

In [ ]:
df_4 = df_4[df_4['처리상태'] == '구매확정'].groupby('주문월').agg({'처리상태': 'count'}).reset_index()
df_4.rename(columns={'처리상태': '구매확정건수'}, inplace=True)
df_4.index = range(1,11)
df_4

#### 다우기술 회사

In [ ]:
df_5 = df_5[df_5['처리상태'] == '구매확정'].groupby('주문월').agg({'처리상태': 'count'}).reset_index()
df_5.rename(columns={'처리상태': '구매확정건수'}, inplace=True)
df_5.index = range(1,12)
df_5 = df_5[:-1]
df_5

#### 천재태블릿 회사

In [ ]:
df_6 = df_6[df_6['처리상태'] == '구매확정'].groupby('주문월').agg({'처리상태': 'count'}).reset_index()
df_6.rename(columns={'처리상태': '구매확정건수'}, inplace=True)
df_6.index = range(1,11)
df_6

#### 그래프로 확인

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams['axes.unicode_minus'] = False
plt.rc('font', family = 'Malgun Gothic')
plt.plot(df_4['주문월'], df_4['구매확정건수'], marker='o', linestyle='-', color='b', label = '지니')
plt.plot(df_5['주문월'], df_5['구매확정건수'], marker='o', linestyle='-', color='r', label = '다우기술')
plt.plot(df_6['주문월'], df_6['구매확정건수'], marker='o', linestyle='-', color='g', label = '천재태블릿')
plt.xticks(range(1, 11), ['1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월', '10월'])
plt.xlabel('주문 월(2022)')
plt.ylabel('구매확정건수')
plt.title('입점업체 월별 거래 빈도 확인')
plt.legend(loc='upper right', bbox_to_anchor=(1.3, 1))

plt.show()

## 2 - a, b번

In [ ]:
df.dtypes

### 회사가 지니일때

In [ ]:
list_mon = [0] * 12

for index, row in df1.iterrows():
    if row['할부기간'] == 0:
        list_mon[int(row['주문월'])-1] += row['순이익']
    else:
        monthly_amount = row['순이익'] / row['할부기간']
        start_month_index = int(row['주문월'])-1
        end_month_index = start_month_index + row['할부기간']
        if end_month_index <= 12:
            for i in range(start_month_index,end_month_index):
                list_mon[i] += monthly_amount
        elif end_month_index <= 24:
            for i in range(start_month_index,12):
                list_mon[i] += monthly_amount
            for i in range(0,(end_month_index-12)):
                list_mon[i] += monthly_amount
        else:
            for i in range(start_month_index,12):
                list_mon[i] += monthly_amount
            for i in range(0,12):
                list_mon[i] += monthly_amount
            for i in range(0,(end_month_index-24)):
                list_mon[i] += monthly_amount  
print(list_mon)
#to do 월별
# 주문월을 확인하고 해당 주문월을 가진 행의 할부기간을 확인한다.
# 할부기간에 맞추어 순이익을 할부기간으로 나누고 나눠진 순이익을 그 다음 월부터 순차적으로 들어갈 수 있도록 한다.
# 12월 다음에는 다시 처음으로 돌아올 수 있도록 한다.
# 그렇게 각각의 리스트 값에 다 더해진 것을 막대그래프로 확인한다.

#to do 년도별
# 주문년도을 확인하고 해당 주문년도을 가진 행의 할부기간을 확인한다.
# 할부기간에 맞추어 순이익을 할부기간으로 나누고 나눠진 순이익을 그 다음 월부터 순차적으로 들어갈 수 있도록 한다.
# 12월 다음에는 다음년도 가격으로 들어갈 수 있도록 한다.
# 그렇게 각각의 리스트 값에 다 더해진 것을 막대그래프로 확인한다.

#월 기간

In [ ]:
import matplotlib.pyplot as plt

plt.bar(range(1, 13), [val / 1e8 for val in list_mon])
plt.xlabel('주문 월')
plt.ylabel('순이익 (억 원)')
plt.title('월별 순이익')
plt.xticks(range(1, 13), ['1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월', '10월', '11월', '12월'])
plt.rc('font', family = 'Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False
plt.show()

In [ ]:
list_year = [0] * 6

for index, row in df1.iterrows():
    if row['할부기간'] == 0:
        list_year[int(row['주문년도'])-2019] += row['순이익']
    else:
        yearly_amount = row['순이익'] / row['할부기간']
        start_month_index = int(row['주문월'])-1
        end_month_index = start_month_index + row['할부기간']
        if end_month_index <= 12:
            for i in range(start_month_index,end_month_index):
                list_year[int(row['주문년도'])-2019] += yearly_amount
        elif end_month_index <= 24:
            for i in range(start_month_index,12):
                list_year[int(row['주문년도'])-2019] += yearly_amount
            for i in range(0,(end_month_index-12)):
                list_year[int(row['주문년도'])-2018] += yearly_amount
        else:
            for i in range(start_month_index,12):
                list_year[int(row['주문년도'])-2019] += yearly_amount
            for i in range(0,12):
                list_year[int(row['주문년도'])-2018] += yearly_amount
            for i in range(0,(end_month_index-24)):
                list_year[int(row['주문년도'])-2017] += yearly_amount  
print(list_year)
#to do 월별
# 주문월을 확인하고 해당 주문월을 가진 행의 할부기간을 확인한다.
# 할부기간에 맞추어 순이익을 할부기간으로 나누고 나눠진 순이익을 그 다음 월부터 순차적으로 들어갈 수 있도록 한다.
# 12월 다음에는 다시 처음으로 돌아올 수 있도록 한다.
# 그렇게 각각의 리스트 값에 다 더해진 것을 막대그래프로 확인한다.

#to do 년도별
# 주문년도을 확인하고 해당 주문년도을 가진 행의 할부기간을 확인한다.
# 할부기간에 맞추어 순이익을 할부기간으로 나누고 나눠진 순이익을 그 다음 월부터 순차적으로 들어갈 수 있도록 한다.
# 12월 다음에는 다음년도 가격으로 들어갈 수 있도록 한다.
# 그렇게 각각의 리스트 값에 다 더해진 것을 막대그래프로 확인한다.

#월 기간

In [ ]:
import matplotlib.pyplot as plt

plt.bar(range(1, 6), [val / 1e8 for val in list_year[1:]])
plt.xlabel('주문 년도')
plt.ylabel('순이익 (억 원)')
plt.title('년도별 순이익')
plt.xticks(range(1, 6), ['2020년', '2021년', '2022년', '2023년', '2024년'])
plt.show()

### 회사가 다우기술일때

In [ ]:
list_mon2 = [0] * 12

for index, row in df2.iterrows():
    if row['할부기간'] == 0:
        list_mon2[int(row['주문월'])-1] += row['순이익']
    else:
        monthly_amount = row['순이익'] / row['할부기간']
        start_month_index = int(row['주문월'])-1
        end_month_index = start_month_index + row['할부기간']
        if end_month_index <= 12:
            for i in range(start_month_index,end_month_index):
                list_mon2[i] += monthly_amount
        elif end_month_index <= 24:
            for i in range(start_month_index,12):
                list_mon2[i] += monthly_amount
            for i in range(0,(end_month_index-12)):
                list_mon2[i] += monthly_amount
        else:
            for i in range(start_month_index,12):
                list_mon2[i] += monthly_amount
            for i in range(0,12):
                list_mon2[i] += monthly_amount
            for i in range(0,(end_month_index-24)):
                list_mon2[i] += monthly_amount  
print(list_mon2)
#to do 월별
# 주문월을 확인하고 해당 주문월을 가진 행의 할부기간을 확인한다.
# 할부기간에 맞추어 순이익을 할부기간으로 나누고 나눠진 순이익을 그 다음 월부터 순차적으로 들어갈 수 있도록 한다.
# 12월 다음에는 다시 처음으로 돌아올 수 있도록 한다.
# 그렇게 각각의 리스트 값에 다 더해진 것을 막대그래프로 확인한다.

#to do 년도별
# 주문년도을 확인하고 해당 주문년도을 가진 행의 할부기간을 확인한다.
# 할부기간에 맞추어 순이익을 할부기간으로 나누고 나눠진 순이익을 그 다음 월부터 순차적으로 들어갈 수 있도록 한다.
# 12월 다음에는 다음년도 가격으로 들어갈 수 있도록 한다.
# 그렇게 각각의 리스트 값에 다 더해진 것을 막대그래프로 확인한다.

#월 기간

In [ ]:
import matplotlib.pyplot as plt

plt.bar(range(1, 13), [val / 1e8 for val in list_mon2])
plt.xlabel('주문 월')
plt.ylabel('순이익 (억 원)')
plt.title('월별 순이익')
plt.xticks(range(1, 13), ['1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월', '10월', '11월', '12월'])
plt.show()

In [ ]:
list_year2 = [0] * 6

for index, row in df2.iterrows():
    if row['할부기간'] == 0:
        list_year2[int(row['주문년도'])-2019] += row['순이익']
    else:
        yearly_amount = row['순이익'] / row['할부기간']
        start_month_index = int(row['주문월'])-1
        end_month_index = start_month_index + row['할부기간']
        if end_month_index <= 12:
            for i in range(start_month_index,end_month_index):
                list_year2[int(row['주문년도'])-2019] += yearly_amount
        elif end_month_index <= 24:
            for i in range(start_month_index,12):
                list_year2[int(row['주문년도'])-2019] += yearly_amount
            for i in range(0,(end_month_index-12)):
                list_year2[int(row['주문년도'])-2018] += yearly_amount
        else:
            for i in range(start_month_index,12):
                list_year2[int(row['주문년도'])-2019] += yearly_amount
            for i in range(0,12):
                list_year2[int(row['주문년도'])-2018] += yearly_amount
            for i in range(0,(end_month_index-24)):
                list_year2[int(row['주문년도'])-2017] += yearly_amount  
print(list_year2)
#to do 월별
# 주문월을 확인하고 해당 주문월을 가진 행의 할부기간을 확인한다.
# 할부기간에 맞추어 순이익을 할부기간으로 나누고 나눠진 순이익을 그 다음 월부터 순차적으로 들어갈 수 있도록 한다.
# 12월 다음에는 다시 처음으로 돌아올 수 있도록 한다.
# 그렇게 각각의 리스트 값에 다 더해진 것을 막대그래프로 확인한다.

#to do 년도별
# 주문년도을 확인하고 해당 주문년도을 가진 행의 할부기간을 확인한다.
# 할부기간에 맞추어 순이익을 할부기간으로 나누고 나눠진 순이익을 그 다음 월부터 순차적으로 들어갈 수 있도록 한다.
# 12월 다음에는 다음년도 가격으로 들어갈 수 있도록 한다.
# 그렇게 각각의 리스트 값에 다 더해진 것을 막대그래프로 확인한다.

#월 기간

In [ ]:
import matplotlib.pyplot as plt

plt.bar(range(1, 6), [val / 1e8 for val in list_year2[1:]])
plt.xlabel('주문 년도')
plt.ylabel('순이익 (억 원)')
plt.title('년도별 순이익')
plt.xticks(range(1, 6), ['2020년', '2021년', '2022년', '2023년', '2024년'])
plt.show()

### 회사가 천재태블릿일때

In [ ]:
list_mon3 = [0] * 12

for index, row in df3.iterrows():
    if row['할부기간'] == 0:
        list_mon3[int(row['주문월'])-1] += row['순이익']
    else:
        monthly_amount = row['순이익'] / row['할부기간']
        start_month_index = int(row['주문월'])-1
        end_month_index = start_month_index + row['할부기간']
        if end_month_index <= 12:
            for i in range(start_month_index,end_month_index):
                list_mon3[i] += monthly_amount
        elif end_month_index <= 24:
            for i in range(start_month_index,12):
                list_mon3[i] += monthly_amount
            for i in range(0,(end_month_index-12)):
                list_mon3[i] += monthly_amount
        else:
            for i in range(start_month_index,12):
                list_mon3[i] += monthly_amount
            for i in range(0,12):
                list_mon3[i] += monthly_amount
            for i in range(0,(end_month_index-24)):
                list_mon3[i] += monthly_amount  
print(list_mon3)
#to do 월별
# 주문월을 확인하고 해당 주문월을 가진 행의 할부기간을 확인한다.
# 할부기간에 맞추어 순이익을 할부기간으로 나누고 나눠진 순이익을 그 다음 월부터 순차적으로 들어갈 수 있도록 한다.
# 12월 다음에는 다시 처음으로 돌아올 수 있도록 한다.
# 그렇게 각각의 리스트 값에 다 더해진 것을 막대그래프로 확인한다.

#to do 년도별
# 주문년도을 확인하고 해당 주문년도을 가진 행의 할부기간을 확인한다.
# 할부기간에 맞추어 순이익을 할부기간으로 나누고 나눠진 순이익을 그 다음 월부터 순차적으로 들어갈 수 있도록 한다.
# 12월 다음에는 다음년도 가격으로 들어갈 수 있도록 한다.
# 그렇게 각각의 리스트 값에 다 더해진 것을 막대그래프로 확인한다.

#월 기간

In [ ]:
import matplotlib.pyplot as plt

plt.bar(range(1, 13), [val / 1e8 for val in list_mon3])
plt.xlabel('주문 월')
plt.ylabel('순이익 (억 원)')
plt.title('월별 순이익')
plt.xticks(range(1, 13), ['1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월', '10월', '11월', '12월'])
plt.show()

In [ ]:
list_year3 = [0] * 6

for index, row in df3.iterrows():
    if row['할부기간'] == 0:
        list_year3[int(row['주문년도'])-2019] += row['순이익']
    else:
        yearly_amount = row['순이익'] / row['할부기간']
        start_month_index = int(row['주문월'])-1
        end_month_index = start_month_index + row['할부기간']
        if end_month_index <= 12:
            for i in range(start_month_index,end_month_index):
                list_year3[int(row['주문년도'])-2019] += yearly_amount
        elif end_month_index <= 24:
            for i in range(start_month_index,12):
                list_year3[int(row['주문년도'])-2019] += yearly_amount
            for i in range(0,(end_month_index-12)):
                list_year3[int(row['주문년도'])-2018] += yearly_amount
        else:
            for i in range(start_month_index,12):
                list_year3[int(row['주문년도'])-2019] += yearly_amount
            for i in range(0,12):
                list_year3[int(row['주문년도'])-2018] += yearly_amount
            for i in range(0,(end_month_index-24)):
                list_year3[int(row['주문년도'])-2017] += yearly_amount  
print(list_year3)
#to do 월별
# 주문월을 확인하고 해당 주문월을 가진 행의 할부기간을 확인한다.
# 할부기간에 맞추어 순이익을 할부기간으로 나누고 나눠진 순이익을 그 다음 월부터 순차적으로 들어갈 수 있도록 한다.
# 12월 다음에는 다시 처음으로 돌아올 수 있도록 한다.
# 그렇게 각각의 리스트 값에 다 더해진 것을 막대그래프로 확인한다.

#to do 년도별
# 주문년도을 확인하고 해당 주문년도을 가진 행의 할부기간을 확인한다.
# 할부기간에 맞추어 순이익을 할부기간으로 나누고 나눠진 순이익을 그 다음 월부터 순차적으로 들어갈 수 있도록 한다.
# 12월 다음에는 다음년도 가격으로 들어갈 수 있도록 한다.
# 그렇게 각각의 리스트 값에 다 더해진 것을 막대그래프로 확인한다.

#월 기간

In [ ]:
import matplotlib.pyplot as plt

plt.bar(range(1, 6), [val / 1e8 for val in list_year3[1:]])
plt.xlabel('주문 년도')
plt.ylabel('순이익 (억 원)')
plt.title('년도별 순이익')
plt.xticks(range(1, 6), ['2020년', '2021년', '2022년', '2023년', '2024년'])
plt.show()

### 세 입점기업을 한 번에 확인하기

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(1, 13), [val / 1e8 for val in list_mon], label = '지니')
plt.plot(range(1, 13), [val / 1e8 for val in list_mon2], label = '다우기술')
plt.plot(range(1, 13), [val / 1e8 for val in list_mon3], label = '천재태블릿')


plt.xlabel('주문 월')
plt.ylabel('순이익 (억 원)')
plt.title('입점 기업별 월별 순이익')
plt.xticks(range(1, 13), ['1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월', '10월', '11월', '12월'])
plt.rc('font', family = 'Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(1, 6), [val / 1e8 for val in list_year[1:]], label = '지니')
plt.plot(range(1, 6), [val / 1e8 for val in list_year2[1:]], label = '다우기술')
plt.plot(range(1, 6), [val / 1e8 for val in list_year3[1:]], label = '천재태블릿')

plt.xlabel('주문 년도')
plt.ylabel('순이익 (억 원)')
plt.title('입점 기업별 연도별 순이익')
plt.xticks(range(1, 6), ['2020년', '2021년', '2022년', '2023년', '2024년'])
plt.legend()
plt.show()

## 3 - a, b번

### 결제 방법과 결제 금액의 연관성 분석

### 결제 방법에 따른 분석

In [ ]:
df.dropna(subset=['결제방법'], inplace=True)
df = df[df['결제방법'] != '-']

In [ ]:
df['결제방법'].unique()
df['판매금액'].fillna(0, inplace=True)

In [ ]:
df['처리상태'].unique()

In [ ]:
df1= df[df['업체명'] == '지니']
df2= df[df['업체명'] == '다우기술']
df3= df[df['업체명'] == '천재태블릿']

In [ ]:
df1

### 지니 회사

In [ ]:
# 우선 등분산성을 확인하자 -> 등분산성이 위배되었다.
df1['결제방법'].unique()

In [ ]:
from scipy.stats import levene

# 등분산성 검정
levene_stat, levene_p = levene(df1['판매금액'][df1['결제방법'] == '신용카드'],
                               df1['판매금액'][df1['결제방법'] == '후불'],
                               df1['판매금액'][df1['결제방법'] == '무통장입금'],
                               df1['판매금액'][df1['결제방법'] == '가상계좌'],
                               df1['판매금액'][df1['결제방법'] == '포인트'],
                               df1['판매금액'][df1['결제방법'] == '적립금'],
                               df1['판매금액'][df1['결제방법'] == '웰컴마일'],
                               df1['판매금액'][df1['결제방법'] == '정기결제'],
                               df1['판매금액'][df1['결제방법'] == '현금간편결제'])

print("Levene's test p-value:", levene_p)

if levene_p < 0.05:
    print("등분산성 가정이 통계적으로 유의미하게 위배되었습니다.")
else:
    print("등분산성 가정이 통계적으로 유의미하게 위배되지 않았습니다.")

In [ ]:
from scipy.stats import f_oneway

result_welch = f_oneway(df1['판매금액'][df1['결제방법'] == '신용카드'],
                               df1['판매금액'][df1['결제방법'] == '후불'],
                               df1['판매금액'][df1['결제방법'] == '무통장입금'],
                               df1['판매금액'][df1['결제방법'] == '가상계좌'],
                               df1['판매금액'][df1['결제방법'] == '포인트'],
                               df1['판매금액'][df1['결제방법'] == '적립금'],
                               df1['판매금액'][df1['결제방법'] == '웰컴마일'],
                               df1['판매금액'][df1['결제방법'] == '정기결제'],
                               df1['판매금액'][df1['결제방법'] == '현금간편결제'])


print("Welch's ANOVA 결과:")
print("F-statistic:", result_welch.statistic)
print("p-value:", result_welch.pvalue)

if result_welch.pvalue < 0.05:
    print("등분산성 가정이 위배되었지만, Welch's ANOVA 결과가 통계적으로 유의미합니다.")
else:
    print("등분산성 가정이 위배되지 않았습니다.")


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

df1['로그_판매금액'] = np.log1p(df1['판매금액'])
plt.figure(figsize=(8, 6))
sns.boxplot(x='결제방법', y='로그_판매금액', data=df1, showfliers=False)
plt.title('지니 회사에서의 결제방법에 따른 결제금액 확인')
plt.rc('font', family = 'Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False
plt.show()


In [ ]:
count_virtual_account = (df1['결제방법'] == '무통장입금').sum()
print(count_virtual_account)

In [ ]:
df1[df1['결제방법'] == '무통장입금']

### 다우기술 회사

In [ ]:
df2['결제방법'].unique()

In [ ]:
from scipy.stats import levene

# 등분산성 검정
levene_stat, levene_p = levene(df2['판매금액'][df2['결제방법'] == '신용카드'],
                               df2['판매금액'][df2['결제방법'] == '포인트'],
                               df2['판매금액'][df2['결제방법'] == '적립금'],
                               df2['판매금액'][df2['결제방법'] == '현금간편결제'])

print("Levene's test p-value:", levene_p)

if levene_p < 0.05:
    print("등분산성 가정이 통계적으로 유의미하게 위배되었습니다.")
else:
    print("등분산성 가정이 통계적으로 유의미하게 위배되지 않았습니다.")

In [ ]:
from scipy.stats import f_oneway

result_welch = f_oneway(df2['판매금액'][df2['결제방법'] == '신용카드'],
                               df2['판매금액'][df2['결제방법'] == '포인트'],
                               df2['판매금액'][df2['결제방법'] == '적립금'],
                               df2['판매금액'][df2['결제방법'] == '현금간편결제'])



print("Welch's ANOVA 결과:")
print("F-statistic:", result_welch.statistic)
print("p-value:", result_welch.pvalue)

if result_welch.pvalue < 0.05:
    print("등분산성 가정이 위배되었지만, Welch's ANOVA 결과가 통계적으로 유의미합니다.")
else:
    print("등분산성 가정이 위배되지 않았습니다.")


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(10, 10))
sns.boxplot(x='결제방법', y='판매금액', data=df2, showfliers=False)
plt.title('다우기술 회사에서의 결제 방법에 따른 결제금액 확인')
plt.rc('font', family = 'Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False
plt.show()


In [ ]:
import numpy as np
plt.boxplot((df2[df2['결제방법'] == '적립금']['판매금액']),
            labels=['적립금'], showfliers=True)

In [ ]:
count_virtual_account = (df2['결제방법'] == '적립금').sum()
print(count_virtual_account)

In [ ]:
df2[df2['결제방법'] == '적립금']['판매금액'].unique()

### 천재태블릿 회사

In [ ]:
df3['결제방법'].unique()

In [ ]:
from scipy.stats import levene

# 등분산성 검정
levene_stat, levene_p = levene(df3['판매금액'][df3['결제방법'] == '신용카드'],
                               df3['판매금액'][df3['결제방법'] == '포인트'],
                               df3['판매금액'][df3['결제방법'] == '적립금'],
                               df3['판매금액'][df3['결제방법'] == '웰컴마일'],
                               df3['판매금액'][df3['결제방법'] == '정기결제'],
                               df3['판매금액'][df3['결제방법'] == '현금간편결제'])

print("Levene's test p-value:", levene_p)

if levene_p < 0.05:
    print("등분산성 가정이 통계적으로 유의미하게 위배되었습니다.")
else:
    print("등분산성 가정이 통계적으로 유의미하게 위배되지 않았습니다.")

In [ ]:
from scipy.stats import f_oneway

result_welch = f_oneway(df3['판매금액'][df3['결제방법'] == '신용카드'],
                               df3['판매금액'][df3['결제방법'] == '포인트'],
                               df3['판매금액'][df3['결제방법'] == '적립금'],
                               df3['판매금액'][df3['결제방법'] == '웰컴마일'],
                               df3['판매금액'][df3['결제방법'] == '정기결제'],
                               df3['판매금액'][df3['결제방법'] == '현금간편결제'])


print("Welch's ANOVA 결과:")
print("F-statistic:", result_welch.statistic)
print("p-value:", result_welch.pvalue)

if result_welch.pvalue < 0.05:
    print("등분산성 가정이 위배되었지만, Welch's ANOVA 결과가 통계적으로 유의미합니다.")
else:
    print("등분산성 가정이 위배되지 않았습니다.")


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(8, 6))
sns.boxplot(x='결제방법', y='판매금액', data=df3, showfliers=False)
plt.title('천재태블릿 회사에서의 결제 방법에 따른 판매금액 확인')
plt.rc('font', family = 'Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False
plt.show()


In [ ]:
count_virtual_account = (df3['결제방법'] == '포인트').sum()
print(count_virtual_account)

In [ ]:
df3[df3['결제방법'] == '포인트']['판매금액'].unique()